# Compare TMA balancing data from previous events with the next events

We are getting ready to balance the telescope twice in the next weeks. First, we will balance the telescope with ComCam and M2 Glass. The M2 Glass and M2 Surrogate have similar weights, with a small difference. We expect the torques applied by the elevation drives will be very close to the previous balancing event(s). A couple of weeks later, we will repeat the procedure with ComCam, M2 Glass, and M1M3 Glass. The M1M3 Glass and M1M3 Cell assembly is much heavier than the M1M3 Mass Simulator (yellow cross) and hundreds of kilograms heavier than the M1M3 Surrogate and M1M3 Cell configuration. This procedure will be much more delicate due to the size and mass of the mirror. 

We want to establish a baseline before we start the procedure, and we need someone to review the data to determine whether we can proceed quickly. 

The links below point to old night logs that might contain useful information. Feel free to unlink them if they are not useful. 

Here is an approximate timeline of different integration phases where we needed to re-balance the telescope. 
We do not necessarily need the whole process. We need the torques once the telescope is already balanced as a baseline.

May to Aug 2023 - M1M3 Surrogate and M1M3 Cell on the TMA

Nov 2023 to Jan 2024 - M1M3 Surrogate and Cell, M2 Surrogate and Cell on the TMA

Feb to Apr 2024 - M2 Surrogate and Cell on the TMA

## Set up notebook

Let's start importing a few libraries to use in this notebook. 

In [ ]:
# Notebook extensions for formatting and auto-reload libraries
%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2

# Standard Python Libraries
import os
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

from astropy.time import Time
from collections import defaultdict
from datetime import datetime, timedelta
from scipy.stats import norm

# LSST Specific Libraries
from lsst_efd_client import EfdClient
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import (
    getCommandsDuringEvent,
    TMAEvent,
    TMAEventMaker,
    TMAState,
)

In [ ]:
async def query_bump_logs_in_chunks(
    start_date,
    end_date,
    client_name="",
    chunk_size_days=3,
    topic_name="lsst.sal.MTM1M3.logevent_logMessage",
    fields=["message"],
):
    """
    Queries the log messages related to bump tests from the EFD in chunks.

    Args:
        start_date (str): Start date of the query in ISO format (YYYY-MM-DD).

        end_date (str): End date of the query in ISO format (YYYY-MM-DD).

        client_name (str, optional): Name of the EFD client. Defaults to "".

        chunk_size_days (int, optional): Number of days per chunk. Defaults to 3.

        topic_name (str, optional): SAL topic name to be queried by the client. Defaults to lsst.sal.MTM1M3.logevent_logMessage.

        fields (list[str], optional): Fields to be queried by the client. Defaults to ["message"].

    Returns:
        pandas.DataFrame: Concatenated DataFrame containing the queried log messages.
    """

    client = makeClient(client_name)

    # Convert start and end dates to datetime objects
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)

    # Initialize an empty DataFrame to store concatenated results
    all_data = pd.DataFrame()

    current_start = start
    while current_start < end:
        current_end = min(current_start + timedelta(days=chunk_size_days), end)
        try:
            # Query the data for the current chunk
            chunk_data = await client.select_time_series(
                topic_name=topic_name,
                fields=fields,
                start=Time(current_start.isoformat(), format="isot", scale="utc"),
                end=Time(current_end.isoformat(), format="isot", scale="utc"),
            )
            # Concatenate the chunk data to the main DataFrame
            all_data = pd.concat([all_data, chunk_data], ignore_index=False)
        except Exception as e:
            print(
                f"Error querying data from {current_start.isoformat()} to {current_end.isoformat()}: {e}"
            )
            continue  # Optionally, continue to the next chunk

        # Move to the next chunk
        current_start = current_end

    return all_data


def makeClient(client_name):
    # Create the client based on client_name
    if client_name == "summit_efd":
        return makeEfdClient("summit_efd")
    elif client_name == "usdf_efd":
        return makeEfdClient("usdf_efd")
    elif client_name == "idf_efd":
        return makeEfdClient("idf_efd")
    else:
        return makeEfdClient()  # Default client


# Example usage:
# begin = "2023-11-13T01:00"
# end = "2023-12-21T01:00"
# bump_logs = await query_bump_logs_in_chunks(begin, end, client_name='')


def showAndClear():
    plt.show()
    # Clear the current axes.
    plt.cla()
    # Clear the current figure.
    plt.clf()
    # Closes all the figure windows.
    plt.close("all")
    plt.close(fig)

    return


async def getDataFrame(client, starts, ends, topic, verbose=True, fields=None):

    all_data = pd.DataFrame()
    for start, end in zip(starts, ends):
        if verbose:
            print(
                r"Starting query for time range {} - {}".format(start, end),
                end=" . . . ",
            )
        if fields != None:
            df_bump = await client.select_time_series(
                topic, fields, Time(start), Time(end)
            )
        else:
            df_bump = await client.select_time_series(
                topic, "*", Time(start), Time(end)
            )

        all_data = pd.concat([all_data, df_bump], ignore_index=False)

        del df_bump

        if verbose:
            print("Finished")

    return all_data


def makeDateRange(startPoint, endPoint, step=np.timedelta64(1, "D")):
    starts = np.arange(startPoint, endPoint, step=step)
    ends = starts + np.timedelta64(1, "D")
    return starts, ends


def fitGaussian(data, ax):
    mu, std = norm.fit(data)

    xmin, xmax = ax.get_xlim()
    x = np.linspace(np.floor(xmin), np.ceil(xmax), int(10e4))
    p = norm.pdf(x, mu, std)

    return mu, std, p, x, xmin, xmax


def getFWHM_from_gaussian(sigma):
    return 2 * np.sqrt(np.log(2) * 2) * sigma

### Setting up the sub-directories

In [ ]:
base_dir = os.getcwd()
figure_dir, data_dir = base_dir + "/SITCOM-1508-plots", base_dir + "/SITCOM-1508-data"
for pathname in [figure_dir, data_dir]:
    if not os.path.isdir(pathname):
        os.mkdir(pathname)

In [ ]:
verbatim = False

## Get all topics, so we can take a look at what is here

In [ ]:
client = makeClient("usdf_efd")

a = await client.get_topics()

if verbatim:
    for entry in a:
        if entry.__contains__("lsst.sal.MTMount"):
            print(entry)

In [ ]:
b = await client.get_fields("lsst.sal.MTMount.elevation")

if verbatim:
    for entry in b:
        print(entry)

## M1M3 Surrogate and M1M3 Cell on the TMA - May 2023-Aug 2023

In [ ]:
startPoint = np.datetime64("2023-05-25T12:00:00")
endPoint = np.datetime64("2023-06-25T12:00:00")
starts, ends = makeDateRange(startPoint, endPoint)

topic_az = "lsst.sal.MTMount.azimuth"
fields_az = ["actualTorque"]

topic_el = "lsst.sal.MTMount.elevation"
fields_el = ["actualPosition"]

In [ ]:
def formatToInt(datetime):
    years = datetime.astype(object).year
    months = datetime.astype(object).month
    if len(str(months)) == 1:
        months = "0" + str(months)
    days = datetime.astype(object).day
    if len(str(days)) == 1:
        days = "0" + str(days)
    return int(str(years) + str(months) + str(days))

In [ ]:
un_count, stopcount, slewcount = 0, 0, 0
new_start_array, new_end_array = (
    [],
    [],
)  # These arrays hold the start and end datestamps where the telescope is not moving
new_start_array.append(starts[0])
for start in starts:
    dayObs = formatToInt(start)
    eventMaker = TMAEventMaker()
    events = eventMaker.getEvents(dayObs)
    for ev in events:
        if ev.type == TMAState.SLEWING or ev.type == TMAState.TRACKING:
            # Fetch beginning and end
            # We want to throw out time between this period, and keep the time between blocks
            slewcount += 1
            new_end_array.append(ev.begin.datetime64)
            new_start_array.append(ev.end.datetime64)
            # print("We want to ditch this time range")
            # print(ev,end='\n\n')
        elif ev.type == TMAState.UNINITIALIZED:
            # TMA is uninitialized, this is an error (probably)
            un_count += 1
            # print("We don't know what is in this data")
            # print(ev,end='\n\n')
        elif ev.type == TMAState.STOPPED:
            # Take start and end of this block
            stopcount += 1
            # print("We want to use this time range")
            # print(ev,end='\n\n')
new_end_array.append(start)
print("Number of uninitialized blocks: {}".format(un_count))
print("Number of slew blocks: {}".format(slewcount))
print("Number of stopped blocks: {}".format(stopcount))

In [ ]:
all_data_az = await getDataFrame(
    client, new_start_array, new_end_array, topic_az, fields=fields_az, verbose=verbatim
)
# all_data_el = await getDataFrame(client,new_start_array,new_end_array,topic_el,fields=fields_el,verbose=verbatim)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=[10, 6])
axs[0].scatter(all_data_az.index, all_data_az)
axs[0].set_ylabel("Torque [N m]")
axs[0].set_xticks(
    np.arange(
        np.min(all_data_az.index.tz_localize(None)),
        np.max(all_data_az.index.tz_localize(None)),
        step=np.timedelta64(5, "D"),
    )
)
axs[0].set_xlim(
    np.min(all_data_az.index.tz_localize(None)),
    np.max(all_data_az.index.tz_localize(None)),
)

axs[1].hist(
    all_data_az["actualTorque"],
    bins=50,
    facecolor="#2ab0ff",
    edgecolor="#169acf",
    linewidth=0.5,
)
mu, std, p, x, xmin, xmax = fitGaussian(all_data_az["actualTorque"], axs[1])
axs[1].plot(
    x,
    p,
    label="$\mu$={:.2E}, $\sigma$={:.2E}".format(mu, std),
    linewidth=2,
    color="red",
)
axs[1].set_ylabel("Counts")
axs[1].set_xlabel("Torque [N m]")
axs[1].legend()
axs[1].set_xlim(xmin, xmax)

for ax in axs:
    ax.grid()

fig.tight_layout()
fig.savefig(figure_dir + "/May23_Aug23.jpg", dpi=200)

showAndClear()

## M1M3 Surrogate and Cell, M2 Surrogate and Cell on the TMA - Nov 2023 to Jan 2024

In [ ]:
startPoint = np.datetime64("2023-11-01T12:00:00")
endPoint = np.datetime64("2024-01-01T12:00:00")
starts, ends = makeDateRange(startPoint, endPoint)

In [ ]:
all_data = await getDataFrame(client,starts,ends,topic,fields=fields,verbose=False)

In [ ]:
lolim,uplim = pd.Timestamp("2023-11-14").tz_localize("UTC"),pd.Timestamp("2023-11-25").tz_localize("UTC")
limited_data = all_data.loc[lolim:uplim]

In [ ]:
fig,axs = plt.subplots(2,1,figsize=[10,6],sharex=True)
for ax in axs:
    ax.plot(limited_data.index,limited_data)
    ax.set_ylabel("Torque [N m]")
    ax.set_xticks(np.arange(lolim,uplim,step=np.timedelta64(1,"D")))
    ax.set_xlim(lolim,uplim)
    ax.grid()
axs[0].set_ylim(-0.4E5,0.4E5)
axs[1].set_ylim(-0.8E4,0.8E4)
axs[1].set_xlabel("Date")
showAndClear()

## M2 Surrogate and Cell on the TMA - Feb to Apr 2024

In [ ]:
startPoint = np.datetime64("2024-02-01T12:00:00")
endPoint = np.datetime64("2024-04-01T12:00:00")
starts, ends = makeDateRange(startPoint, endPoint)

In [ ]:
all_data = await getDataFrame(client,starts,ends,topic,fields=fields,verbose=False)

In [ ]:
lolim,uplim = pd.Timestamp("2024-02-05").tz_localize("UTC"),pd.Timestamp("2024-02-15").tz_localize("UTC")
limited_data = all_data.loc[lolim:uplim]

In [ ]:
fig,axs = plt.subplots(2,1,figsize=[10,6],sharex=True)
for ax in axs:
    ax.plot(limited_data.index,limited_data)
    ax.set_ylabel("Torque [N m]")
    ax.set_xticks(np.arange(lolim,uplim,step=np.timedelta64(2,"D")))
    ax.set_xlim(lolim,uplim)
    ax.grid()
axs[0].set_ylim(-2E5,2E5)
axs[1].set_ylim(-0.8E2,0.8E2)
axs[1].set_xlabel("Date")
showAndClear()